### DSPy Experiments and Examples

In [1]:
# Importing the required libraries
import dspy
import helper_dspy
import ollama

# Ignoring the warnings
import warnings
warnings.filterwarnings('ignore')

### Basic examples

In [23]:
# Use my helper function to initiate DSPy clients for GPT-4, GPT-35-Turbo, and Mixtral-8x7b
mixtral = helper_dspy.get_dspy_client('mixtral')
gpt35turbo = helper_dspy.get_dspy_client('gpt_35_turbo')
gpt4 = helper_dspy.get_dspy_client('gpt_4')

# Create a json object for the DSPy clients
dspy_clients_json = {
    "mixtral": mixtral,
    "gpt_35_turbo": gpt35turbo,
    "gpt_4": gpt4
}

active_model = gpt4

In [3]:
# Setup dspy to use GPT-4 as the default language model
dspy.settings.configure(lm=active_model)

# While you can access the model direclty like this:
#
#    response = gpt4("It's hump day.  What day of the week do you think it is?")
#
# It's better to use the various modules that are available in the dspy library like below where we use the Predict module.
#

predict_module = dspy.Predict("user_input -> model_output")
print(predict_module)

prediction = predict_module(user_input="Who is the fastest man and woman in the world?")

# The outupt is a Predection class and you can access the response using the response1 and response2 attributes
print(prediction)
print("\n")
print(prediction.model_output)


Predict(StringSignature(user_input -> model_output
    instructions='Given the fields `user_input`, produce the fields `model_output`.'
    user_input = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'User Input:', 'desc': '${user_input}'})
    model_output = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Model Output:', 'desc': '${model_output}'})
))
Prediction(
    model_output="The fastest man in the world is Usain Bolt, with a record-breaking 100m sprint time of 9.58 seconds. The fastest woman in the world is Elaine Thompson-Herah, who set the women's 100m world record at 10.54 seconds."
)


The fastest man in the world is Usain Bolt, with a record-breaking 100m sprint time of 9.58 seconds. The fastest woman in the world is Elaine Thompson-Herah, who set the women's 100m world record at 10.54 seconds.


In [19]:
# See the history for the model
active_model.inspect_history()

# You can also use the dspy library to access the history of the mode





Classify provided content about AI into one or more of the following topics: Regulation, Policy, Security, Privacy, Ethics, Use Cases, Workforce, Trends

---

Follow the following format.

Article: content about AI to be classified
Topics: is of one or more of the following topics: Regulation, Policy, Security, Privacy, Ethics, Use Cases, Workforce, Trends

---

Article: The AI arms race between the US and China is heating up Tech billionaires aren’t the only ones fighting for control of AI. The US and China are too. The two world superpowers are competing with each other for everything from the intellectual know-how to design AI hardware and software to the raw materials that power artificial intelligence systems. Both are also using government subsidies to spur new advancements. Where the US holds a commanding lead at the moment is the development of generative AI systems such as large language models (LLMs), according to Frank Long with Goldman Sachs' (GS) Office of Applied Inno

### Multiple models with context

In [5]:
user_input = "Who is the world's fastest man?"
predict_module = dspy.Predict("user_input -> model_output", n=2)

# Iterate over the dspy clients json and get the predictions
for key, value in dspy_clients_json.items():
    with dspy.context(lm=value):
        print("=====================================================================================================")
        print(f"Model: {key.upper()}")
        print("-----------------------------------------------------------------------------------------------------")
        prediction = predict_module(user_input=user_input)
        for response in prediction.completions:
            print(response.model_output)
            print("\n")
            print("-------------")
            print("\n")


Model: MIXTRAL
-----------------------------------------------------------------------------------------------------
Model Output: The world's fastest man, according to the Guinness World Records, is Usain Bolt from Jamaica. He set the world record in the 100 meters race with a time of 9.58 seconds at the World Athletics Championships in Berlin, Germany, in 2009.


-------------


Model Output: The world's fastest man is Usain Bolt, according to the Guinness World Records. He set the world record for the 100 meters at 9.58 seconds during the 2009 World Athletics Championships in Berlin, Germany. However, if we consider superheroes, then The Flash from DC Comics is often depicted as the fastest man alive.


-------------


Model: GPT_35_TURBO
-----------------------------------------------------------------------------------------------------
Usain Bolt is considered the world's fastest man.


-------------


Usain Bolt is considered the world's fastest man.


-------------


Model: GPT

### Create your own class (module) that implements a custom signature.

#### Create a class that classifies the topics for a given article.

- **Regulation**: Laws and guidelines governing AI use.

- **Policy**: Strategies, policies, and frameworks shaping AI deployment.

- **Security**: Keeping AI systems secure against threats.

- **Privacy**: Protecting individual privacy in the context of AI applications.

- **Ethics**: The ethical considerations and impacts of AI technologies.

- **Use Cases**: Real-world examples of AI implementation in the public sector.

- **Workforce**: Preparing the public sector workforce for AI through education and training.

- **Trends**: The latest trends in AI technology and their potential implications.

In [25]:

class TopicClassifier(dspy.Signature):
    """Classify provided content about AI into one or more of the following topics: Regulation, Policy, Security, Privacy, Ethics, Use Cases, Workforce, Trends"""
    article = dspy.InputField(desc="content about AI to be classified")
    topics = dspy.OutputField(desc=" must match one or more of the following topics: Regulation, Policy, Security, Privacy, Ethics, Use Cases, Workforce, Trends")

In [26]:
# Instatiate the TopicClassifier class
topic_classifier = dspy.Predict(TopicClassifier)    
print(topic_classifier)

Predict(TopicClassifier(article -> topics
    instructions='Classify provided content about AI into one or more of the following topics: Regulation, Policy, Security, Privacy, Ethics, Use Cases, Workforce, Trends'
    article = Field(annotation=str required=True json_schema_extra={'desc': 'content about AI to be classified', '__dspy_field_type': 'input', 'prefix': 'Article:'})
    topics = Field(annotation=str required=True json_schema_extra={'desc': ' must match one or more of the following topics: Regulation, Policy, Security, Privacy, Ethics, Use Cases, Workforce, Trends', '__dspy_field_type': 'output', 'prefix': 'Topics:'})
))


In [31]:

article = """
The AI arms race between the US and China is heating up
Tech billionaires aren’t the only ones fighting for control of AI. The US and China are too.

The two world superpowers are competing with each other for everything from the intellectual know-how to design AI hardware and software to the raw materials that power artificial intelligence systems. Both are also using government subsidies to spur new advancements.

Where the US holds a commanding lead at the moment is the development of generative AI systems such as large language models (LLMs), according to Frank Long with Goldman Sachs' (GS) Office of Applied Innovation. These models hoover up existing data and use it as the basis for chatbots like Open AI's ChatGPT.

Another advantage for the US is that it can place export restrictions on high-performance semiconductors designed by companies like Nvidia (NVDA) that are in intense demand across the AI world. That, for now, is keeping the development of the most sophisticated LLMs out of China’s reach, Long said.

FILE - A sign for a Nvidia building is shown in Santa Clara, Calif., May 31, 2023. The chipmaker posted stronger-than-expected results for its latest quarter and provided further evidence that the excitement surrounding artificial intelligence likely won't subside anytime soon. (AP Photo/Jeff Chiu, File)
Nvidia is now one of the world's most valuable companies due to soaring demand for AI computing power. (Jeff Chiu/AP Photo, File) (ASSOCIATED PRESS)
China, however, is countering with its own maneuvers. It is restricting the export of chipmaking metals gallium and germanium to the US while also reportedly amassing a new $27 billion chip fund to back its own major projects.

The fierce competition for AI talent between the two countries turned up a notch this past week when the US Justice Department unsealed an indictment charging a Chinese national and former Google (GOOG, GOOGL) AI software developer with stealing 500 files of confidential code that the tech giant uses for its supercomputing data centers to train LLMs.

The government alleged that after stealing the intellectual property, the defendant began simultaneously working for rival companies in China.

The charges “are the latest illustration of the lengths affiliates of companies based in the People’s Republic of China are willing to go to steal American innovation,” FBI director Christopher Wray said in a statement.

'Horse race'
The fight for global supremacy of AI was a hot topic of discussion earlier this month at the Web Summit’s annual technology conference in Doha, Qatar, an event that attracted investors and tech executives from around the world.

AI leaders from the public and private sectors acknowledged that the US and China currently have the edge. The countries currently rank No. 1 and No. 2 in Tortoise Media's Global AI Index, which measures nations based on AI investment, innovation, and implementation.

But the leaders also said at the summit that it's too soon to know which countries, over time, will leverage the technology to the greatest economic and societal advantage. Singapore, for example, is now No. 3 in Tortoise's index after moving up rapidly in recent years.

A general view shows the financial business district in Singapore on March 19, 2019. - Paris has climbed to the top of the world's priciest city for expatriates, tied first with Singapore and Hong Kong according to a survey on March 19 that named the capital of strife-torn Venezuela as the cheapest. (Photo by Roslan RAHMAN / AFP)        (Photo credit should read ROSLAN RAHMAN/AFP via Getty Images)
Singapore is climbing the ranks of AI powers. (Photo by Roslan RAHMAN / AFP) (ROSLAN RAHMAN via Getty Images)
"I think it's not going to be as straightforward of a horse race — this person or that person, this country or that country," Long said. "It's going to be a full stack with participants in competition" for the energy, computing power, data, and models needed for AI systems.

Long and his team at Goldman said in a recent white paper they suspect certain geopolitical “swing states” — like the United Kingdom, the United Arab Emirates (UAE), Israel, Japan, the Netherlands, and South Korea — may be best positioned to tap the technology and form AI alliances.

Other Asian countries beyond China already have several advantages. Taiwan is home to a cutting-edge semiconductor manufacturer, Taiwan Semiconductor Manufacturing Company (TSM), which produces 90% of the world’s most advanced semiconductors and 68% of semiconductors worldwide.

Engineers work in a cleanroom at the Taiwan Semiconductor Research Institute in Hsinchu, Taiwan, February 10, 2022. Picture taken February 10, 2022. Picture taken February 10, 2022. REUTERS/Ann Wang
Engineers work in a cleanroom at the Taiwan Semiconductor Research Institute in Hsinchu, Taiwan. (Ann Wang/REUTERS) (REUTERS / Reuters)
Japan and South Korea are also home to leading semiconductor manufacturing and design firms, and they are setting aside more government funds to drive AI advancement.

Japan allocated $13 billion to the technology in its 2023 budget, up from $8.6 billion in 2022, while South Korea committed $470 billion over the next 23 years to create the world’s largest semiconductor manufacturing hub.

Other potential AI hubs could emerge in Europe and the Middle East. The Netherlands — home to ASML (ASML) — is already the world’s sole manufacturer of ultraviolet lithography machines, which are required to fabricate leading-edge semiconductors.

An employees make his way in a laboratory at ASML, a Dutch company which is currently the largest supplier in the world of semiconductor manufacturing machines via photolithography systems in Veldhoven on April 17, 2018. - They call it 
An employee in a laboratory at ASML, a Dutch company that is currently the largest supplier in the world of semiconductor manufacturing machines via photolithography systems. (EMMANUEL DUNAND / AFP via Getty Images) (EMMANUEL DUNAND via Getty Images)
The UAE has a $10 billion fund to invest in late-state technologies, while Israel has also attracted billions in private AI investments.

Global power will shift towards nations that produce versus solely consume AI technology, according to Alaa Abdulaal, head of digital foresight for the Riyadh-based multilateral foundation Digital Cooperation Organization (DCO). The group's mission is to recommend government policies that promote access to technology.

"It cannot be only done by government itself," Abdulaal said. "It needs to take a cooperative approach, where we have at the same table the private sector, public sector, civil society — all of them sitting together to come up with the right set of frameworks for AI."

'Getting much closer'
Today, most countries have adopted, at minimum, a national strategy to develop and protect against the potential risks of AI. And some have cleared the way for regulations, though none have been tested against market forces.

In March, the EU is expected to adopt new legal restraints around AI. Earlier this year, the bloc's member states signaled their agreement with the AI Act, the world’s first comprehensive legislation to regulate the technology.

Federal laws specific to AI don’t exist yet in the US or UK, and it’s unknown whether that will happen.

In October, President Biden issued an executive order to encourage safe AI development, including privacy protections. More than a dozen US states have adopted multiple AI-related laws.

FILE - President Joe Biden speaks about government regulations on artificial intelligence systems during an event in the East Room of the White House, Oct. 30, 2023, in Washington. The White House said Wednesday, Feb. 21, 2024, that it is seeking public comment on the risks and benefits of having an AI system's key components publicly available for anyone to use and modify. (AP Photo/Evan Vucci, File)
President Joe Biden speaks about government regulations on artificial intelligence systems during an event in the East Room of the White House last year. (Evan Vucci/AP Photo, File) (ASSOCIATED PRESS)
The UK, for its part, rolled out a “National AI Strategy” in 2022 and committed roughly $4 billion in chip development subsidies. The nation has also attracted the third-largest level of private investment in AI, behind the US and China.

State subsidies and export bans may not hold sway forever as nations compete, said Jihad Tayara, CEO of Evoteq, a Dubai-based company that facilitates public-private collaborations to build AI into public infrastructure.

"It's getting much closer," Tayara said of the digital divide, noting that most nations have access to high-speed 5G data capabilities.

And human capital, along with access to open-source models, will matter too.

"Today we're seeing open-source models emerging that are in many use cases equivalent in capability to the absolute most frontier models," Long said.

Nations that prepare their workforces to develop and implement AI models will gain a considerable edge, Abdulaal said.

"This opportunity cannot be seen unless we have the right people in place."
"""
    
with dspy.context(lm=gpt35turbo):
    classification = topic_classifier(article=article)
    print(classification.topics)

Trends, Policy, Workforce, Ethics, Security


In [ ]:
import dspy
import helper_dspy
from dspy import teleprompt
from dspy.retrieve.chromadb_rm import ChromadbRM
# from index_bridge_world_system import CHROMADB_DIR, CHROMA_COLLECTION_NAME

import json


class SymbolsToResearch(dspy.Module):
    """
    Extract relevant ticker symbols for analyst doing research
    """
    def __init__(self):
        super().__init__()
        self.entity_extractor = dspy.Predict("instruction, current_thread -> symbols")

    def forward(self, current_thread):
        meta_prompt = f"Your job is to analyze the sequence of user questions and deduce the ticker symbols of companies (if any) that the last question applies to. Be clever, sometimes you will have to deduce the companies from the question because they won't be named. Return only the ticker symbols."
        prediction = self.entity_extractor(
            current_thread=f"{current_thread}"
        )
        answer = prediction.symbols
        if "Symbols: " in answer:
            start = answer.rindex("Symbols: ") + len("Symbols: ")
            answer = answer[start:]
        return [a.strip() for a in answer.split(',')]
    

In [ ]:
# Setup DSP
gpt_35_symbol_finder = helper_dspy.get_dspy_client(service_id='gpt_35_turbo', max_tokens=500, temperature=0.0, top_p=0.2)
dspy.settings.configure(lm=gpt_35_symbol_finder)


# Test question
examples = """
Your job is to analyze the sequence of user questions and deduce the ticker symbols of companies (if any) that the last question applies to. Be clever, sometimes you will have to deduce the companies from the question because they won't be named. Return only the ticker symbols.

# Example 1:
Sequence_of_user_questions: 
    - What are the current earnings per share for the big 3 US cloud providers?
Symbols: MSFT,AMZN,GOOG

# Example 2:
Sequence_of_user_questions: 
    - What are the current earnings per share for the big 3 US cloud providers?
    - Can you tell me the definition of opex?
    - Do Tesla and Apple have debt?
Symbols: TSLA,AAPL

# Example 3:
Sequence_of_user_questions: 
    - Did Bank of America increase their earnings last year?
    - How about Wells Fargo?
Symbols: WFC

===

Analyze the sequence of user questions and deduce the ticker symbols of companies (if any) that the last question applies to. Be clever, sometimes you will have to deduce the companies from the question because they won't be named. Return the ticker symbols.

Sequence_of_user_questions:
    - Tell me about the risk of the major US banks.
Symbols: 
"""
module = SymbolsToResearch()
response = module(examples)
print(response)

gpt_35_symbol_finder.history[0]

In [ ]:
from dspy.teleprompt import BootstrapFewShot

# Small training set with question and answer pairs
trainset = [dspy.Example(question="What were the two main things the author worked on before college?", 
                         answer="Writing and programming").with_inputs('question'),
            dspy.Example(question="What kind of writing did the author do before college?", 
                         answer="Short stories").with_inputs('question'),
            ...
            ]

# The teleprompter will bootstrap missing labels: reasoning chains and retrieval contexts
teleprompter = BootstrapFewShot(metric=dspy.evaluate.answer_exact_match)
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

Below is user input to an AI.  

**User inputs**  
Graph the current earnings per share for the US regional banks from 2020 to 2024.

TASK:  
1. Reivew the input and determine if the AI should lookup SEC filings for specific companies to respond to the input.
2. Reframe the input to include the list of companies that might be helpful or reframe the input so it is clear for the AI.
3. Return the reframed input.



===


Below is a sequence of user inputs to an AI.  

**Sequence of user inputs**  
- Can you graph the current earnings per share for the big 3 US cloud providers (Amazon, Microsoft, Google) from 2020 to 2024?
- include apple too

TASK:  

Assume the last input is a follow-up to the previous ones. Reframe the last question so it can stand on its own and return the reframed input.

USER INPUT TO AI:
Graph the current earnings per share for the US regional banks from 2020 to 2023.

TASK:  
Return a comma separated list of ticker symbols the AI should lookup SEC filings for to aid in its response to the input.

RESPONSE FORMAT:
Comma separated list
